In [20]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [47]:
import requests
import io
import urllib.parse
import json

#Command to start the DB sever and restore from file
"""sudo docker run \
 -p 9000:9000  -p 9009:9009 \
 -p 8812:8812 -p 9003:9003 \
 -v "/questdb_data:/root/.questdb/" questdb/questdb"""
#sudo docker logs --follow CONTAINER

'sudo docker run  -p 9000:9000  -p 9009:9009  -p 8812:8812 -p 9003:9003  -v "/questdb_data:/root/.questdb/" questdb/questdb'

In [100]:
#For creating the table initially
q = 'create table forecasts'\
    '(location string,'\
    'Date string,'\
    'snowfall float,'\
    'maxTemp float,'\
    'maxWind float,'\
    'sun_percent float)'
r = requests.get("http://localhost:9000/exec?query=" + q)
print(r.status_code)

200


In [5]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_driver = os.path.join(os.getcwd(), "chromedriver")
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)

In [101]:
class location:
    def __init__(self,name,lat,long):
        self.name=name
        self.lat=lat
        self.long=long
    
    def get_forecast(self):
        url="https://spotwx.com/products/grib_index.php?model=gem_lam_continental&lat="+str(self.lat)+"&lon="+str(self.long)+"&tz=America/Vancouver&display=table"
        driver.get(url)
        driver.execute_script("$('.buttons-csv').click();")
        time.sleep(2)
        df=pd.read_csv("SpotWx.csv")
        os.remove("SpotWx.csv")
        self.forecast=df
        return df
    
    def weather_synopsis(self,df):
        new_snow=df["SQP"].iloc[-1] #get last accumulated snow
        melting=df["TMP"].max()>0
        wind=df["WS"].max()
        #sun during the day hours only
        cloud=df.CLOUD.mask((df.TIME.str.slice(stop=2).astype(int)<8) | (df.TIME.str.slice(stop=2).astype(int)>20))
        sun=100-cloud.mean()
        data = {'New Snow':  [new_snow],
                'Melting': [melting],
                'Max Winds': [wind],
                'Weighted Sun':[sun]
               }
        result = pd.DataFrame(data, index=[self.name])
        return result
    
    def store_today(self,df):
        #Check to see if we already have the data
        today=df.iloc[0:df[df.TIME == '00:00'].index[0]]
        q = "select count"\
        " from forecasts"\
        " where date ='"+today.DATE + "' and location ='"+self.name+"'"
        query = urllib.parse.quote(q[0])
        r = requests.get("http://localhost:9000/exec?query="+query)
        if (json.loads(r.text)['dataset'][0][0]==0):
            cloud=today.CLOUD.mask((today.TIME.str.slice(stop=2).astype(int)<8) | (today.TIME.str.slice(stop=2).astype(int)>20))
            sun=100-cloud.mean()
            date=today.DATE[0]
            query = "insert into forecasts values('"\
            + self.name + "','"\
            + date + "',"\
            + str(today["SQP"].iloc[-1]) + ","\
            + str(today.TMP.max()) + ","\
            + str(today.WS.max()) + ","\
            + str(sun) + ")"
            r = requests.get("http://localhost:9000/exec?query=" + query)

            if (r.status_code==200):
                return True
            else:
                return False

In [102]:
blackcomb=location("black",50.09970, -122.86943)

garibaldi=location("gar",49.93299, -123.03600)

joffre_lake=location("jof",50.34143, -122.47768)

s2s=location("s2s",9.65218, -123.09211)

cloudburst=location("cloud",49.93298, -123.23602)

manning=location("man",49.06805, -120.91646)

coq=location("coq",49.61094, -121.05758)

rogers=location("rog",51.30123, -117.52014)

sites=[blackcomb,garibaldi,joffre_lake,cloudburst,manning,coq,rogers]

In [103]:
view=pd.DataFrame(columns = ['New Snow', 'Melting','Max Winds','Weighted Sun'])

In [104]:
print("Grabbing site data...")
for site in sites:
    forecast=site.get_forecast()
    if(site.store_today(forecast)):
        print(site.name)
    result=site.weather_synopsis(forecast)
    view=view.append(result)

Grabbing site data...
{'query': "select count from forecasts where date ='2022/02/22' and location ='black'", 'columns': [{'name': 'count', 'type': 'LONG'}], 'dataset': [[0]], 'count': 1}
black
{'query': "select count from forecasts where date ='2022/02/22' and location ='gar'", 'columns': [{'name': 'count', 'type': 'LONG'}], 'dataset': [[0]], 'count': 1}
gar
{'query': "select count from forecasts where date ='2022/02/22' and location ='jof'", 'columns': [{'name': 'count', 'type': 'LONG'}], 'dataset': [[0]], 'count': 1}
jof
{'query': "select count from forecasts where date ='2022/02/22' and location ='cloud'", 'columns': [{'name': 'count', 'type': 'LONG'}], 'dataset': [[0]], 'count': 1}
cloud
{'query': "select count from forecasts where date ='2022/02/22' and location ='man'", 'columns': [{'name': 'count', 'type': 'LONG'}], 'dataset': [[0]], 'count': 1}
man
{'query': "select count from forecasts where date ='2022/02/22' and location ='coq'", 'columns': [{'name': 'count', 'type': 'LONG'

In [20]:
view
view.style.background_gradient(cmap="RdYlGn",subset=["New Snow"]) #style snow column

,New Snow,Melting,Max Winds,Weighted Sun
black,1.100000,False,9,73.814815
gar,1.100000,False,21,79.000000
jof,0.900000,False,12,75.777778
cloud,0.100000,False,8,86.703704
man,1.400000,False,13,78.703704
coq,0.200000,False,13,73.333333
rog,1.900000,False,11,76.962963


In [21]:
view.style.background_gradient(cmap="cividis",subset=["Weighted Sun"],vmin=50) #style sun column

,New Snow,Melting,Max Winds,Weighted Sun
black,1.100000,False,9,73.814815
gar,1.100000,False,21,79.000000
jof,0.900000,False,12,75.777778
cloud,0.100000,False,8,86.703704
man,1.400000,False,13,78.703704
coq,0.200000,False,13,73.333333
rog,1.900000,False,11,76.962963


In [19]:
requests.get("http://localhost:9000/exec?query= BACKUP table forecasts;")

'{"query":"","error":"No query text","position":0}'

{'query': "select CASE WHEN EXISTS(select 1 from forecasts where date ='2022-02-05' and location ='rog')THEN (1)ELSE (0) END",
 'error': 'unknown function name: EXISTS(CURSOR)',
 'position': 17}

In [190]:
driver.quit()